# Início

In [5]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torchsummary import summary

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
import time
import os
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [18]:
path = '../standardized_view'
dados = []
for files in os.listdir(path):
    dados.append(pd.read_csv(os.path.join(path, files), header=None))

len(dados)

IsADirectoryError: [Errno 21] Is a directory: '../standardized_view/UCI'

In [20]:

actis = [s[:-4] for s in os.listdir(path)]
actis

['', 'W', 'K', 'RealWorld_w', 'MotionS', 'RealWorld_t']

In [5]:
aux = dados[0].values
N = aux.shape[0]
X_train = np.array([aux[i*60:(i+1)*60,:].T for i in range(N//60)])

In [6]:
X_train.shape, N

((8614, 3, 60), 516840)

In [7]:
input_shape = X_train.T.shape[:-1]
input_shape

(60, 3)

In [8]:
generated_sample_shape = input_shape
noise_shape = 100
input_layer = noise_shape

n_layers = 4
kernel_size = 10
strides = 2
g_units_base=32

steps = generated_sample_shape[0]
layer_steps = [steps]
for i in range(n_layers):
    layer_steps.append(int(np.ceil(float(layer_steps[-1]) / float(strides))))
layer_steps.reverse()

conv_units = []
if n_layers > 1:
    conv_units.append(g_units_base)
    for _ in range(n_layers - 2):  # minus the first and the last layers
        conv_units.append(conv_units[-1] * 2)
conv_units.reverse()
# the last layer must be aligned to the number of dimensions of input.
conv_units.append(generated_sample_shape[-1])

In [9]:
conv_units, layer_steps

([128, 64, 32, 3], [4, 8, 15, 30, 60])

In [10]:
class Genera(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(input_layer, layer_steps[0] * conv_units[0] * 2, device=device)
        self.l2 = nn.BatchNorm1d(layer_steps[0] * conv_units[0] * 2, device=device)
        self.l3 = nn.ConvTranspose1d(conv_units[0] * 2, conv_units[0], kernel_size,
                                    stride=strides, padding=4, device=device)
        self.l4 = nn.BatchNorm1d(conv_units[0], device=device)
        self.l5 = nn.ConvTranspose1d(conv_units[0], conv_units[1], kernel_size,
                                    stride=strides, padding=4, device=device)
        self.l6 = nn.BatchNorm1d(conv_units[1], device=device)
        self.l7 = nn.ConvTranspose1d(conv_units[1], conv_units[2], kernel_size,
                                    stride=strides, padding=4, device=device)
        self.l8 = nn.BatchNorm1d(conv_units[2], device=device)
        self.l9 = nn.ConvTranspose1d(conv_units[2], conv_units[3], kernel_size,
                                    stride=strides, padding=6, output_padding=0, device=device)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = F.relu(x)
        x = x.reshape(x.shape[0], layer_steps[0], conv_units[0] * 2)
        x = torch.transpose(x, 1, 2)
        x = self.l3(x)
        x = self.l4(x)
        x = F.relu(x)
        x = self.l5(x)
        x = self.l6(x)
        x = F.relu(x)
        x = self.l7(x)
        x = self.l8(x)
        x = F.relu(x)
        x = self.l9(x)
        return x

In [11]:
generator = Genera().to(device)
summary(generator, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]         103,424
       BatchNorm1d-2                 [-1, 1024]           2,048
   ConvTranspose1d-3               [-1, 128, 8]         327,808
       BatchNorm1d-4               [-1, 128, 8]             256
   ConvTranspose1d-5               [-1, 64, 16]          81,984
       BatchNorm1d-6               [-1, 64, 16]             128
   ConvTranspose1d-7               [-1, 32, 32]          20,512
       BatchNorm1d-8               [-1, 32, 32]              64
   ConvTranspose1d-9                [-1, 3, 60]             963
Total params: 537,187
Trainable params: 537,187
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 2.05
Estimated Total Size (MB): 2.11
-------------------------------------------

In [12]:
units = [32]
for _ in range(n_layers - 1):  # exclude the first layer.
    units.append(units[-1] * 2)

In [13]:
units

[32, 64, 128, 256]

In [14]:
class Discri(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Conv1d(input_shape[-1], units[0], kernel_size,stride=strides,padding=6, device=device)
        self.l2 = nn.Conv1d(units[0], units[1], kernel_size,stride=strides,padding=4, device=device)
        self.l3 = nn.Conv1d(units[1], units[2], kernel_size,stride=strides,padding=4, device=device)
        self.l4 = nn.BatchNorm1d(units[2], device=device)
        self.l5 = nn.Conv1d(units[2], units[3], kernel_size,stride=strides,padding=4, device=device)
        self.l6 = nn.BatchNorm1d(units[3], device=device)
        self.l7 = nn.Linear(layer_steps[0] * units[-1], 1, device=device)

    def forward(self, x):
        x = self.l1(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l2(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l3(x)
        x = self.l4(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l5(x)
        x = self.l6(x)
        x = F.leaky_relu(x, 0.2)
        x = torch.transpose(x, 1, 2)
        x = torch.flatten(x, start_dim=1)
        x = self.l7(x)
        return x
discriminator = Discri().to(device)

In [15]:
summary(discriminator, (3,60))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 32]             992
            Conv1d-2               [-1, 64, 16]          20,544
            Conv1d-3               [-1, 128, 8]          82,048
       BatchNorm1d-4               [-1, 128, 8]             256
            Conv1d-5               [-1, 256, 4]         327,936
       BatchNorm1d-6               [-1, 256, 4]             512
            Linear-7                    [-1, 1]           1,025
Total params: 433,313
Trainable params: 433,313
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 1.65
Estimated Total Size (MB): 1.70
----------------------------------------------------------------


In [16]:
generator.l1.weight = nn.init.trunc_normal_(generator.l1.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l1.bias.data.zero_()
generator.l2.weight = nn.init.trunc_normal_(generator.l2.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l2.bias.data.zero_()
generator.l3.weight = nn.init.trunc_normal_(generator.l3.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l3.bias.data.zero_()
generator.l4.weight = nn.init.trunc_normal_(generator.l4.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l4.bias.data.zero_()
generator.l5.weight = nn.init.trunc_normal_(generator.l5.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l5.bias.data.zero_()
generator.l6.weight = nn.init.trunc_normal_(generator.l6.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l6.bias.data.zero_()
generator.l7.weight = nn.init.trunc_normal_(generator.l7.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l7.bias.data.zero_()
generator.l8.weight = nn.init.trunc_normal_(generator.l8.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l8.bias.data.zero_()
generator.l9.weight = nn.init.trunc_normal_(generator.l9.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l9.bias.data.zero_()

tensor([0., 0., 0.])

In [17]:
discriminator.l1.weight = nn.init.trunc_normal_(discriminator.l1.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l1.bias.data.zero_()
discriminator.l2.weight = nn.init.trunc_normal_(discriminator.l2.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l2.bias.data.zero_()
discriminator.l3.weight = nn.init.trunc_normal_(discriminator.l3.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l3.bias.data.zero_()
discriminator.l4.weight = nn.init.trunc_normal_(discriminator.l4.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l4.bias.data.zero_()
discriminator.l5.weight = nn.init.trunc_normal_(discriminator.l5.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l5.bias.data.zero_()
discriminator.l6.weight = nn.init.trunc_normal_(discriminator.l6.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l6.bias.data.zero_()
discriminator.l7.weight = nn.init.trunc_normal_(discriminator.l7.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l7.bias.data.zero_()

tensor([0.])

# Treinamento

In [18]:
X_train = torch.tensor(X_train.astype(np.float32), dtype=torch.float32, device=device).detach()

In [19]:
generator = generator.to(device)
discriminator = discriminator.to(device)

# torch.manual_seed(42)

loss_fc = nn.BCEWithLogitsLoss()
sig = nn.Sigmoid()

g_opt = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_opt = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

n_epochs = 300   # number of epochs to run
batch_size = 128  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

dLoss = []
gLoss = []
acc = []

z_batch = torch.randn(X_train.shape[0], noise_shape, device=device).detach()
generated_samples = generator(z_batch)
fake_logits = discriminator(generated_samples)
real_logits = discriminator(X_train)
real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
d_loss = real_loss + fake_loss
g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
gLoss.append(float(g_loss))
dLoss.append(float(d_loss))

real_prob = sig(real_logits)
fake_prob = sig(fake_logits)
y_real = torch.ones_like(real_prob)
y_fake = torch.zeros_like(fake_prob)
y = torch.cat((y_real, y_fake), axis=0)
y_pred = torch.cat((real_prob, fake_prob), axis=0)
acc.append(float(torch.mean(((y_pred>.5).double() == y).double())))

# training loop
for epoch in range(n_epochs):
    generator.train()
    discriminator.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:,:]
            z_batch = torch.randn(batch_size, noise_shape, device=device).detach()
            # generator pass
            generated_samples = generator(z_batch)
            fake_logits = discriminator(generated_samples)
            g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
            g_opt.zero_grad()
            g_loss.backward()
            g_opt.step()
            # discriminator pass
            generated_samples = generator(z_batch)
            fake_logits = discriminator(generated_samples)
            real_logits = discriminator(X_batch)
            # check acc
            real_prob = sig(real_logits)
            fake_prob = sig(fake_logits)
            y_real = torch.ones_like(real_prob)
            y_fake = torch.zeros_like(fake_prob)
            y = torch.cat((y_real, y_fake), axis=0)
            y_pred = torch.cat((real_prob, fake_prob), axis=0)
            if torch.mean(((y_pred>.5).double() == y).double()) <= 0.75:
                real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
                fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
                d_loss = real_loss + fake_loss
                d_opt.zero_grad()
                d_loss.backward()
                d_opt.step()
            # print progress
            bar.set_postfix(loss=float(g_loss))
    generator.eval()
    discriminator.eval()
    z_batch = torch.randn(X_train.shape[0], noise_shape, device=device).detach()
    generated_samples = generator(z_batch)
    fake_logits = discriminator(generated_samples)
    real_logits = discriminator(X_train)
    real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
    fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
    d_loss = real_loss + fake_loss
    g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
    gLoss.append(float(g_loss))
    dLoss.append(float(d_loss))
    real_prob = sig(real_logits)
    fake_prob = sig(fake_logits)
    y_real = torch.ones_like(real_prob)
    y_fake = torch.zeros_like(fake_prob)
    y = torch.cat((y_real, y_fake), axis=0)
    y_pred = torch.cat((real_prob, fake_prob), axis=0)
    acc.append(float(torch.mean(((y_pred>.5).double() == y).double())))

Epoch 299: 100%|██████████| 68/68 [00:01<00:00, 57.02batch/s, loss=0.787]


In [21]:
def train_GAN(generator, discriminator, X_train, n_epochs, batch_size):
    loss_fc = nn.BCEWithLogitsLoss()
    sig = nn.Sigmoid()

    g_opt = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    d_opt = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    batch_start = torch.arange(0, len(X_train), batch_size)

    dLoss = []
    gLoss = []
    acc = []

    z_batch = torch.randn(X_train.shape[0], noise_shape, device=device).detach()
    generated_samples = generator(z_batch)
    fake_logits = discriminator(generated_samples)
    real_logits = discriminator(X_train)
    real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
    fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
    d_loss = real_loss + fake_loss
    g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
    gLoss.append(float(g_loss))
    dLoss.append(float(d_loss))

    real_prob = sig(real_logits)
    fake_prob = sig(fake_logits)
    y_real = torch.ones_like(real_prob)
    y_fake = torch.zeros_like(fake_prob)
    y = torch.cat((y_real, y_fake), axis=0)
    y_pred = torch.cat((real_prob, fake_prob), axis=0)
    acc.append(float(torch.mean(((y_pred>.5).double() == y).double())))

    # training loop
    for epoch in range(n_epochs):
        generator.train()
        discriminator.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size,:,:]
                z_batch = torch.randn(batch_size, noise_shape, device=device).detach()
                # generator pass
                generated_samples = generator(z_batch)
                fake_logits = discriminator(generated_samples)
                g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
                g_opt.zero_grad()
                g_loss.backward()
                g_opt.step()
                # discriminator pass
                generated_samples = generator(z_batch)
                fake_logits = discriminator(generated_samples)
                real_logits = discriminator(X_batch)
                # check acc
                real_prob = sig(real_logits)
                fake_prob = sig(fake_logits)
                y_real = torch.ones_like(real_prob)
                y_fake = torch.zeros_like(fake_prob)
                y = torch.cat((y_real, y_fake), axis=0)
                y_pred = torch.cat((real_prob, fake_prob), axis=0)
                if torch.mean(((y_pred>.5).double() == y).double()) <= 0.75:
                    real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
                    fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
                    d_loss = real_loss + fake_loss
                    d_opt.zero_grad()
                    d_loss.backward()
                    d_opt.step()
                # print progress
                bar.set_postfix(loss=float(g_loss))
        generator.eval()
        discriminator.eval()
        z_batch = torch.randn(X_train.shape[0], noise_shape, device=device).detach()
        generated_samples = generator(z_batch)
        fake_logits = discriminator(generated_samples)
        real_logits = discriminator(X_train)
        real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
        fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
        d_loss = real_loss + fake_loss
        g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
        gLoss.append(float(g_loss))
        dLoss.append(float(d_loss))
        real_prob = sig(real_logits)
        fake_prob = sig(fake_logits)
        y_real = torch.ones_like(real_prob)
        y_fake = torch.zeros_like(fake_prob)
        y = torch.cat((y_real, y_fake), axis=0)
        y_pred = torch.cat((real_prob, fake_prob), axis=0)
        acc.append(float(torch.mean(((y_pred>.5).double() == y).double())))
    return dLoss, gLoss, acc

In [22]:
PATH = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/tcgan/'
torch.save(generator.state_dict(), PATH+'generator_'+actis[0]+'.pkl')
torch.save(discriminator.state_dict(), PATH+'discriminator_'+actis[0]+'.pkl')

In [20]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=dLoss, mode="lines", showlegend=True, name='dLoss'))
fig.add_trace(go.Scatter(y=gLoss, mode="lines", showlegend=True, name='gLoss'))
fig.add_trace(go.Scatter(y=acc, mode="lines", showlegend=True, name='acc'))
fig.update_layout(width=1000, height=600)

fig.show()

In [23]:
z_batch = torch.randn(batch_size, noise_shape, device=device).detach()
generated_samples = generator(z_batch).cpu().detach().numpy()

In [24]:
i = 0
i += 1
px.line(generated_samples[i,:,:].T)

In [25]:
aux = np.array([aux[i*60:(i+1)*60,:].T for i in range(N//60)])

In [26]:
px.line(aux[i,:,:].T)

# Treinando todos os modelos

In [30]:
PATH = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/tcgan/'
for j in range(len(actis)):
    generator = Genera().to(device)
    discriminator = Discri().to(device)
    aux = dados[j].values
    N = aux.shape[0]
    X_train = np.array([aux[i*60:(i+1)*60,:].T for i in range(N//60)])
    X_train = torch.tensor(X_train.astype(np.float32), dtype=torch.float32, device=device).detach()
    dLoss, gLoss, acc = train_GAN(generator, discriminator, X_train, 300, 128)
    torch.save(generator.state_dict(), PATH+'generator_'+actis[j]+'.pkl')
    torch.save(discriminator.state_dict(), PATH+'discriminator_'+actis[j]+'.pkl')

Epoch 299: 100%|██████████| 51/51 [00:00<00:00, 57.41batch/s, loss=1.07]


In [33]:
j = 1
generator = Genera().to(device)
discriminator = Discri().to(device)
generator.load_state_dict(torch.load(PATH+'generator_'+actis[j]+'.pkl', weights_only=True))
generator.eval()
discriminator.load_state_dict(torch.load(PATH+'discriminator_'+actis[j]+'.pkl', weights_only=True))
discriminator.eval()

Discri(
  (l1): Conv1d(3, 32, kernel_size=(10,), stride=(2,), padding=(6,))
  (l2): Conv1d(32, 64, kernel_size=(10,), stride=(2,), padding=(4,))
  (l3): Conv1d(64, 128, kernel_size=(10,), stride=(2,), padding=(4,))
  (l4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l5): Conv1d(128, 256, kernel_size=(10,), stride=(2,), padding=(4,))
  (l6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l7): Linear(in_features=1024, out_features=1, bias=True)
)

# Metrics

In [18]:
"""Reimplement TimeGAN-pytorch Codebase.

Reference: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar,
"Time-series Generative Adversarial Networks,"
Neural Information Processing Systems (NeurIPS), 2019.

Paper link: https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks

Last updated Date: October 18th 2021
Code author: Zhiwei Zhang (bitzzw@gmail.com)

-----------------------------

utils.py

(1) train_test_divide: Divide train and test data for both original and synthetic data.
(2) extract_time: Returns Maximum sequence length and each sequence length.
(3) random_generator: random vector generator
(4) NormMinMax: return data info
"""


def train_test_divide (data_x, data_x_hat, data_t, data_t_hat, train_rate = 0.8):
  """Divide train and test data for both original and synthetic data.

  Args:
    - data_x: original data
    - data_x_hat: generated data
    - data_t: original time
    - data_t_hat: generated time
    - train_rate: ratio of training data from the original data
  """
  # Divide train/test index (original data)
  no = len(data_x)
  print("data_x:",no)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]

  train_x = [data_x[i] for i in train_idx]
  test_x = [data_x[i] for i in test_idx]
  train_t = [data_t[i] for i in train_idx]
  test_t = [data_t[i] for i in test_idx]

  # Divide train/test index (synthetic data)
  no = len(data_x_hat)
  print("data_x_hat:",no)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]

  train_x_hat = [data_x_hat[i] for i in train_idx]
  test_x_hat = [data_x_hat[i] for i in test_idx]
  train_t_hat = [data_t_hat[i] for i in train_idx]
  test_t_hat = [data_t_hat[i] for i in test_idx]

  return train_x, train_x_hat, test_x, test_x_hat, train_t, train_t_hat, test_t, test_t_hat


def extract_time (data):
  """Returns Maximum sequence length and each sequence length.

  Args:
    - data: original data

  Returns:
    - time: extracted time information
    - max_seq_len: maximum sequence length
  """
  time = list()
  max_seq_len = 0
  for i in range(len(data)):
    max_seq_len = max(max_seq_len, len(data[i][:,0]))
    time.append(len(data[i][:,0]))

  return time, max_seq_len


def random_generator (batch_size, z_dim, T_mb, max_seq_len):
  """Random vector generation.

  Args:
    - batch_size: size of the random vector
    - z_dim: dimension of random vector
    - T_mb: time information for the random vector
    - max_seq_len: maximum sequence length

  Returns:
    - Z_mb: generated random vector
  """
  Z_mb = list()
  for i in range(batch_size):
    temp = np.zeros([max_seq_len, z_dim])
    temp_Z = np.random.uniform(0., 1, [T_mb[i], z_dim])
    temp[:T_mb[i],:] = temp_Z
    Z_mb.append(temp_Z)
  return Z_mb


def NormMinMax(data):
    """Min-Max Normalizer.

    Args:
      - data: raw data

    Returns:
      - norm_data: normalized data
      - min_val: minimum values (for renormalization)
      - max_val: maximum values (for renormalization)
    """
    min_val = np.min(np.min(data, axis=0), axis=0)
    data = data - min_val  # [3661, 24, 6]

    max_val = np.max(np.max(data, axis=0), axis=0)
    norm_data = data / (max_val + 1e-7)

    return norm_data, min_val, max_val

In [19]:
"""Reimplement TimeGAN-pytorch Codebase.

Reference: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar,
"Time-series Generative Adversarial Networks,"
Neural Information Processing Systems (NeurIPS), 2019.

Paper link: https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks

Last updated Date: October 18th 2021
Code author: Zhiwei Zhang (bitzzw@gmail.com)

-----------------------------

data.py

(0) MinMaxScaler: Min Max normalizer
(1) sine_data_generation: Generate sine dataset
(2) real_data_loading: Load and preprocess real data
  - stock_data: https://finance.yahoo.com/quote/GOOG/history?p=GOOG
  - energy_data: http://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction
(3) load_data: download or generate data
(4): batch_generator: mini-batch generator
"""

from os.path import dirname, abspath


def MinMaxScaler(data):
  """Min Max normalizer.

  Args:
    - data: original data

  Returns:
    - norm_data: normalized data
  """
  numerator = data - np.min(data, 0)
  denominator = np.max(data, 0) - np.min(data, 0)
  norm_data = numerator / (denominator + 1e-7)
  return norm_data


def sine_data_generation (no, seq_len, dim):
  """Sine data generation.

  Args:
    - no: the number of samples
    - seq_len: sequence length of the time-series
    - dim: feature dimensions

  Returns:
    - data: generated data
  """
  # Initialize the output
  data = list()

  # Generate sine data
  for i in range(no):
    # Initialize each time-series
    temp = list()
    # For each feature
    for k in range(dim):
      # Randomly drawn frequency and phase
      freq = np.random.uniform(0, 0.1)
      phase = np.random.uniform(0, 0.1)

      # Generate sine signal based on the drawn frequency and phase
      temp_data = [np.sin(freq * j + phase) for j in range(seq_len)]
      temp.append(temp_data)

    # Align row/column
    temp = np.transpose(np.asarray(temp))
    # Normalize to [0,1]
    temp = (temp + 1)*0.5
    # Stack the generated data
    data.append(temp)

  return data


def real_data_loading (data_name, seq_len):
  """Load and preprocess real-world datasets.

  Args:
    - data_name: stock or energy
    - seq_len: sequence length

  Returns:
    - data: preprocessed data.
  """
  assert data_name in ['stock','energy']

  if data_name == 'stock':
    ori_data = np.loadtxt(dirname(dirname(abspath(__file__))) + '/data/stock_data.csv', delimiter = ",",skiprows = 1)
  elif data_name == 'energy':
    ori_data = np.loadtxt(dirname(dirname(abspath(__file__))) + '/data/energy_data.csv', delimiter = ",",skiprows = 1)

  # Flip the data to make chronological data
  ori_data = ori_data[::-1]
  # Normalize the data
  ori_data = MinMaxScaler(ori_data)

  # Preprocess the dataset
  temp_data = []
  # Cut data by sequence length
  for i in range(0, len(ori_data) - seq_len):
    _x = ori_data[i:i + seq_len]
    temp_data.append(_x)

  # Mix the datasets (to make it similar to i.i.d)
  idx = np.random.permutation(len(temp_data))
  data = []
  for i in range(len(temp_data)):
    data.append(temp_data[idx[i]])

  return data


def load_data(opt):
  ## Data loading
  if opt.data_name in ['stock', 'energy']:
    ori_data = real_data_loading(opt.data_name, opt.seq_len)  # list: 3661; [24,6]
  elif opt.data_name == 'sine':
    # Set number of samples and its dimensions
    no, dim = 10000, 5
    ori_data = sine_data_generation(no, opt.seq_len, dim)
  print(opt.data_name + ' dataset is ready.')

  return ori_data


def batch_generator(data, time, batch_size):
  """Mini-batch generator.

  Args:
    - data: time-series data
    - time: time information
    - batch_size: the number of samples in each batch

  Returns:
    - X_mb: time-series data in each batch
    - T_mb: time information in each batch
  """
  no = len(data)
  idx = np.random.permutation(no)
  train_idx = idx[:batch_size]

  X_mb = list(data[i] for i in train_idx)
  T_mb = list(time[i] for i in train_idx)

  return X_mb, T_mb

In [34]:
"""Reimplement TimeGAN-pytorch Codebase.

Reference: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar,
"Time-series Generative Adversarial Networks,"
Neural Information Processing Systems (NeurIPS), 2019.

Paper link: https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks

Last updated Date: October 18th 2021
Code author: Zhiwei Zhang (bitzzw@gmail.com)

-----------------------------

predictive_metrics.py

Note: Use post-hoc RNN to classify original data and synthetic data

Output: discriminative score (np.abs(classification accuracy - 0.5))
"""

# Necessary Packages
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from sklearn.metrics import accuracy_score
tf1.disable_v2_behavior()


def discriminative_score_metrics (ori_data, generated_data,batch_size):
  """Use post-hoc RNN to classify original data and synthetic data

  Args:
    - ori_data: original data
    - generated_data: generated synthetic data

  Returns:
    - discriminative_score: np.abs(classification accuracy - 0.5)
  """
  # Initialization on the Graph
  tf1.reset_default_graph()
  tf.random.set_seed(1234)
  tf1.set_random_seed(1234)


  # Basic Parameters
  no, seq_len, dim = np.asarray(ori_data).shape

  # Set maximum sequence length and each sequence length
  ori_time, ori_max_seq_len = extract_time(ori_data)
  generated_time, generated_max_seq_len = extract_time(generated_data)
  max_seq_len = max([ori_max_seq_len, generated_max_seq_len])

  ## Builde a post-hoc RNN discriminator network
  # Network parameters
  hidden_dim = int(dim/2)
  iterations = 5000
  batch_size = 128

  # Input place holders
  # Feature
  X = tf1.placeholder(tf.float32, [None, max_seq_len, dim], name = "myinput_x")
  X_hat = tf1.placeholder(tf.float32, [None, max_seq_len, dim], name = "myinput_x_hat")

  T = tf1.placeholder(tf.int32, [None], name = "myinput_t")
  T_hat = tf1.placeholder(tf.int32, [None], name = "myinput_t_hat")

  # discriminator function
  def discriminator (x, t):
    """Simple discriminator function.

    Args:
      - x: time-series data
      - t: time information

    Returns:
      - y_hat_logit: logits of the discriminator output
      - y_hat: discriminator output
      - d_vars: discriminator variables
    """
    with tf1.variable_scope("discriminator", reuse = tf1.AUTO_REUSE) as vs:
      d_cell = tf1.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh, name = 'd_cell')
      d_outputs, d_last_states = tf1.nn.dynamic_rnn(d_cell, x, dtype=tf.float32, sequence_length = t)
      # y_hat_logit = tf1.contrib.layers.fully_connected(d_last_states, 1, activation_fn=None)
      y_hat_logit = tf1.layers.dense(d_last_states, 1, activation=None)
      y_hat = tf.nn.sigmoid(y_hat_logit)
      d_vars = [v for v in tf1.all_variables() if v.name.startswith(vs.name)]

    return y_hat_logit, y_hat, d_vars

  y_logit_real, y_pred_real, d_vars = discriminator(X, T)
  y_logit_fake, y_pred_fake, _ = discriminator(X_hat, T_hat)

  # Loss for the discriminator
  d_loss_real = tf1.reduce_mean(tf1.nn.sigmoid_cross_entropy_with_logits(logits = y_logit_real,
                                                                       labels = tf1.ones_like(y_logit_real)))
  d_loss_fake = tf1.reduce_mean(tf1.nn.sigmoid_cross_entropy_with_logits(logits = y_logit_fake,
                                                                       labels = tf1.zeros_like(y_logit_fake)))
  d_loss = d_loss_real + d_loss_fake

  # optimizer
  d_solver = tf1.train.AdamOptimizer().minimize(d_loss, var_list = d_vars)

  ## Train the discriminator
  # Start session and initialize
  sess = tf1.Session()
  sess.run(tf1.global_variables_initializer())

  # Train/test division for both original and generated data
  train_x, train_x_hat, test_x, test_x_hat, train_t, train_t_hat, test_t, test_t_hat = \
  train_test_divide(ori_data, generated_data, ori_time, generated_time)


  # Training step
  for itt in tqdm(range(iterations)):

    # Batch setting
    X_mb, T_mb = batch_generator(train_x, train_t, batch_size)
    X_hat_mb, T_hat_mb = batch_generator(train_x_hat, train_t_hat, batch_size)

    # Train discriminator
    _, step_d_loss = sess.run([d_solver, d_loss],
                              feed_dict={X: X_mb, T: T_mb, X_hat: X_hat_mb, T_hat: T_hat_mb})
  print('# %d Loss: %f'% (itt,step_d_loss))
  print('Real:',len(X_mb))
  print('Fake:',len(X_hat_mb))

  ## Test the performance on the testing set
  d_s=[]
  for i in  tqdm(range(iterations)):
      y_pred_real_curr, y_pred_fake_curr = sess.run([y_pred_real, y_pred_fake],
                                                    feed_dict={X: test_x, T: test_t, X_hat: test_x_hat, T_hat: test_t_hat})

      y_pred_final = np.squeeze(np.concatenate((y_pred_real_curr, y_pred_fake_curr), axis = 0))
      y_label_final = np.concatenate((np.ones([len(y_pred_real_curr),]), np.zeros([len(y_pred_fake_curr),])), axis = 0)

      # Compute the accuracy
      acc = accuracy_score(y_label_final, (y_pred_final>0.5))
      discriminative_score = np.abs(0.5-acc)
      d_s.append(discriminative_score)
  #return y_pred_real_curr.shape, y_pred_fake_curr.shape,y_pred_final.shape,y_label_final.shape
  data=np.array(d_s)

  return np.mean(data),np.var(data),np.std(data)


In [35]:
'''
2019 NeurIPS Submission
Title: Time-series Generative Adversarial Networks
Authors: Jinsung Yoon, Daniel Jarrett, Mihaela van der Schaar

Last Updated Date: May 29th 2019
Code Author: Jinsung Yoon (jsyoon0823@gmail.com)

-----------------------------

Predictive_Score_Metrics
- Use Post-hoc RNN to predict one-step ahead (last feature)

Inputs
- dataX: Original data
- dataX_hat: Synthetic ata

Outputs
- Predictive Score (MAE of one-step ahead prediction)
'''

#%% Necessary Packages
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import numpy as np
from sklearn.metrics import mean_absolute_error

#%% Post-hoc RNN one-step ahead predictor

def predictive_score_metrics (dataX, dataX_hat,batch_size):

    # Initialization on the Graph
    tf1.reset_default_graph()

    # Basic Parameters
    No = len(dataX)
    data_dim = len(dataX[0][0,:])

    # Maximum seq length and each seq length
    dataT = list()
    Max_Seq_Len = 0
    for i in range(No):
        Max_Seq_Len = max(Max_Seq_Len, len(dataX[i][:,0]))
        dataT.append(len(dataX[i][:,0]))

    # Network Parameters
    hidden_dim = max(int(data_dim/2),1)
    iterations = 5000
    #batch_size = 128

    #%% input place holders

    X = tf1.placeholder(tf.float32, [None, Max_Seq_Len-1, data_dim-1], name = "myinput_x")
    T = tf1.placeholder(tf.int32, [None], name = "myinput_t")
    Y = tf1.placeholder(tf.float32, [None, Max_Seq_Len-1, 1], name = "myinput_y")

    #%% builde a RNN discriminator network

    def predictor (X, T):

        with tf1.variable_scope("predictor", reuse = tf1.AUTO_REUSE) as vs:

            d_cell = tf1.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh, name = 'd_cell')

            d_outputs, d_last_states = tf1.nn.dynamic_rnn(d_cell, X, dtype=tf.float32, sequence_length = T)

            #Y_hat = tf.contrib.layers.fully_connected(d_outputs, 1, activation_fn=None)
            Y_hat = tf1.layers.dense(d_outputs, 1, activation=None)

            Y_hat_Final = tf.nn.sigmoid(Y_hat)

            d_vars = [v for v in tf1.all_variables() if v.name.startswith(vs.name)]

        return Y_hat_Final, d_vars

    #%% Functions
    # Variables
    Y_pred, d_vars = predictor(X, T)

    # Loss for the predictor
    D_loss = tf1.losses.absolute_difference(Y, Y_pred)

    # optimizer
    D_solver = tf1.train.AdamOptimizer().minimize(D_loss, var_list = d_vars)

    #%% Sessions

    # Session start
    sess = tf1.Session()
    sess.run(tf1.global_variables_initializer())

    # Training using Synthetic dataset
    for itt in tqdm(range(iterations)):

        # Batch setting
        idx = np.random.permutation(len(dataX_hat))
        train_idx = idx[:batch_size]

        X_mb = list(dataX_hat[i][:-1,:(data_dim-1)] for i in train_idx)
        T_mb = list(dataT[i]-1 for i in train_idx)
        Y_mb = list(np.reshape(dataX_hat[i][1:,(data_dim-1)],[len(dataX_hat[i][1:,(data_dim-1)]),1]) for i in train_idx)

        # Train discriminator
        _, step_d_loss = sess.run([D_solver, D_loss], feed_dict={X: X_mb, T: T_mb, Y: Y_mb})

        #%% Checkpoints
#        if itt % 500 == 0:
#            print("[step: {}] loss - d loss: {}".format(itt, np.sqrt(np.round(step_d_loss,4))))

    #%% Use Original Dataset to test

    # Make Batch with Original Data
    idx = np.random.permutation(len(dataX_hat))
    train_idx = idx[:No]

    X_mb = list(dataX[i][:-1,:(data_dim-1)] for i in train_idx)
    T_mb = list(dataT[i]-1 for i in train_idx)
    Y_mb = list(np.reshape(dataX[i][1:,(data_dim-1)], [len(dataX[i][1:,(data_dim-1)]),1]) for i in train_idx)

    # Predict Fugure
    pred_Y_curr = sess.run(Y_pred, feed_dict={X: X_mb, T: T_mb})

    # Compute MAE
    MAE_Temp = 0
    M_list=[]
    for i in range(No):
        MAE_Temp = MAE_Temp + mean_absolute_error(Y_mb[i], pred_Y_curr[i,:,:])
        M_list.append(mean_absolute_error(Y_mb[i], pred_Y_curr[i,:,:]))

    MAE = MAE_Temp / No
    data=np.array(M_list)

    return np.mean(data),np.var(data),


# Script metrics

In [22]:
print(tf.__version__)

2.13.1


In [ ]:
# !pip install tensorflow==2.13.1

In [26]:
generator.load_state_dict(torch.load(PATH+'generator_'+actis[j]+'.pkl', map_location=torch.device('cpu')))

<ipython-input-26-04092124ea9e>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(PATH+'generator_'+actis[j]+'.pkl', map_location=torch.d

<All keys matched successfully>

In [36]:
PATH = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/tcgan/'
ds = []
ps = []
for j in range(len(actis)):
    print(j)
    generator = Genera().to(device)
    # generator.load_state_dict(torch.load(PATH+'generator_'+actis[j]+'.pkl', weights_only=True))
    generator.load_state_dict(torch.load(PATH+'generator_'+actis[j]+'.pkl', map_location=torch.device('cpu')))
    generator.eval()
    aux = dados[j].values
    N = aux.shape[0]
    X_train = np.array([aux[i*60:(i+1)*60,:].T for i in range(N//60)])
    X_train = torch.tensor(X_train.astype(np.float32), dtype=torch.float32, device=device).detach()
    batch_size = 128
    z_batch = torch.randn(batch_size, noise_shape, device=device).detach()
    generated_samples = generator(z_batch)
    real_sample = torch.swapaxes(X_train,1,2).cpu().detach().numpy()
    synthetic_sample = torch.swapaxes(generated_samples,1,2).cpu().detach().numpy()
    med, var, std = discriminative_score_metrics(real_sample, synthetic_sample,batch_size)
    ds.append(med)
    med, var = predictive_score_metrics(real_sample, synthetic_sample,batch_size)
    ps.append(med)
print(ds,ps)

0


<ipython-input-36-019a0e9451f9>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(PATH+'generator_'+actis[j]+'.pkl', map_location=torch.d

data_x: 8614
data_x_hat: 8614


100%|██████████| 5000/5000 [01:57<00:00, 42.64it/s]


# 4999 Loss: 0.937215
Real: 128
Fake: 128


...

data_x: 6427
data_x_hat: 6427


100%|██████████| 5000/5000 [01:59<00:00, 41.83it/s]


# 4999 Loss: 0.386073
Real: 128
Fake: 128


100%|██████████| 5000/5000 [01:41<00:00, 49.07it/s]
<ipython-input-35-409ea1845e05>:64: UserWarning: `tf.nn.rnn_cell.GRUCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.GRUCell`, and will be replaced by that in Tensorflow 2.0.
  d_cell = tf1.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh, name = 'd_cell')
<ipython-input-35-409ea1845e05>:69: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  Y_hat = tf1.layers.dense(d_outputs, 1, activation=None)
100%|██████████| 5000/5000 [12:52<00:00,  6.47it/s]


[0.18572257690075444, 0.24600246002460027, 0.38827626233313983, 0.42687173534532796, 0.3083203732503889, 0.432348367029549] [0.086212285, 3.332347, 0.1221192, 1.7915704, 1.4256096, 1.4900668]
